In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target
df.head()

In [ ]:
df.info()
df.describe()


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='target', y='alcohol', data=df)
plt.title('Alcohol Content by Wine Class')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x='alcohol', y='color_intensity', hue='target', data=df)
plt.title('Alcohol vs Color Intensity by Class')
plt.show()

In [ ]:
class PrismRules:
    def __init__(self):
        self.rules = []

    def fit(self, X, y):
        data = pd.concat([X, y], axis=1)
        class_labels = y.unique()

        for c in class_labels:
            subset = data[data[y.name] == c]
            rest = data[data[y.name] != c]
            while not subset.empty:
                best_rule = None
                best_acc = 0
                conditions = []
                remaining_features = list(X.columns)

                while remaining_features:
                    best_feature, best_value, best_acc_tmp = None, None, 0
                    for f in remaining_features:
                        for val in X[f].unique():
                            covered = subset[subset[f] == val]
                            acc = len(covered) / (len(X[X[f] == val]) + 1e-6)
                            if acc > best_acc_tmp:
                                best_feature, best_value, best_acc_tmp = f, val, acc
                    if best_feature is None:
                        break
                    conditions.append((best_feature, best_value))
                    remaining_features.remove(best_feature)
                    subset = subset[subset[best_feature] == best_value]
                    if len(subset) == 0:
                        break
                self.rules.append((conditions, c))

    def predict(self, X):
        preds = []
        for _, row in X.iterrows():
            found = False
            for conds, label in self.rules:
                if all(row[f] == v for f, v in conds):
                    preds.append(label)
                    found = True
                    break
            if not found:
                preds.append(-1)
        return preds

In [ ]:
X = df.drop(columns='target')
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

prism = PrismRules()
prism.fit(X_train, y_train)

for rule in prism.rules:
    print(rule)

In [ ]:
y_pred = prism.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
new_data = pd.DataFrame({
   'alcohol': [14.0, 14.00],
   'malic_acid': [2.0, 2.0],
   'ash': [2.3, 2.2],
   'alcalinity_of_ash': [19.0, 11.0],
   'magnesium': [95.0, 95.0],
   'total_phenols': [2.2, 2.5],
   'flavanoids': [0.14, 0.5],
   'nonflavanoid_phenols': [0.14, 0.5],
   'proanthocyanins': [1.6, 1.5],
   'color_intensity': [7.0, 6.0],
   'hue': [0.7, 0.6],
   'od280/od315_of_diluted_wines': [3.2, 3.0],
   'proline': [550.0, 1400.0]
})